# Global Food Prices

Understanding how xxxxx



___________________________________________
Author: Andreia Ferreira Almeida

In [18]:
# Imports
import os
import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings('ignore')

### Loading data

In [6]:
proj_dir = os.path.dirname(os.path.abspath("wfp_market_food_prices.csv"))
data = os.path.join(proj_dir, "wfp_market_food_prices.csv")
print(data)

C:\kaggel\kaggel_tests\wfp_market_food_prices.csv


In [7]:
#Loading Data
gfp = pd.read_csv(data, engine='python')
gfp.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,1,2014,50.0,WFP
1,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,2,2014,50.0,WFP
2,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,3,2014,50.0,WFP
3,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,4,2014,50.0,WFP
4,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,5,2014,50.0,WFP


In [88]:
gfp.shape

(743914, 11)

### Data Treating

In [8]:
#rename«ing cols
gfp.columns = ['country_id', 'country_name', 'locality_id', 'locality_name', 'market_id', 'market_name', 'commodity_purchase_id', 'commodity_purchased', 'currency_id', 'name_of_currency', 'market_type_id', 'market_type', 'measurement_id', 'unit_of_goods_measurement', 'month_recorded', 'year_recorded','mp_price:_price_paid', 'source_supplying_price_information']
gfp.columns = [x.upper() for x in gfp.columns]

In [10]:
gfp.head()

,COUNTRY_ID,COUNTRY_NAME,LOCALITY_ID,LOCALITY_NAME,MARKET_ID,MARKET_NAME,COMMODITY_PURCHASE_ID,COMMODITY_PURCHASED,CURRENCY_ID,NAME_OF_CURRENCY,MARKET_TYPE_ID,MARKET_TYPE,MEASUREMENT_ID,UNIT_OF_GOODS_MEASUREMENT,MONTH_RECORDED,YEAR_RECORDED,MP_PRICE:_PRICE_PAID,SOURCE_SUPPLYING_PRICE_INFORMATION
0,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,1,2014,50.0,WFP
1,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,2,2014,50.0,WFP
2,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,3,2014,50.0,WFP
3,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,4,2014,50.0,WFP
4,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,5,2014,50.0,WFP


In [23]:
#droping id cols in order to simplify the data visualization
#id_final = [col if col.endswith('_ID') else None for col in gfp.columns]
#df_without_id = 
#df_without_id = gfp.drop(['COUNTRY_ID','LOCALITY_ID',  ])

id_final = [col for col in gfp.columns if col.endswith('_ID')]

In [24]:
id_final

['COUNTRY_ID',
 'LOCALITY_ID',
 'MARKET_ID',
 'COMMODITY_PURCHASE_ID',
 'CURRENCY_ID',
 'MARKET_TYPE_ID',
 'MEASUREMENT_ID']

In [26]:
gfp.head()

,COUNTRY_ID,COUNTRY_NAME,LOCALITY_ID,LOCALITY_NAME,MARKET_ID,MARKET_NAME,COMMODITY_PURCHASE_ID,COMMODITY_PURCHASED,CURRENCY_ID,NAME_OF_CURRENCY,MARKET_TYPE_ID,MARKET_TYPE,MEASUREMENT_ID,UNIT_OF_GOODS_MEASUREMENT,MONTH_RECORDED,YEAR_RECORDED,MP_PRICE:_PRICE_PAID,SOURCE_SUPPLYING_PRICE_INFORMATION
0,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,1,2014,50.0,WFP
1,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,2,2014,50.0,WFP
2,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,3,2014,50.0,WFP
3,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,4,2014,50.0,WFP
4,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,5,2014,50.0,WFP


In [27]:
gfp.drop(id_final, axis=1, inplace=True) #inputing axis due to the list of cols

In [193]:
gfp.head()

,COUNTRY_NAME,LOCALITY_NAME,MARKET_SHOPS,COMMODITY_PURCHASED,NAME_OF_CURRENCY,MARKET_TYPE,UNIT_OF_GOODS_MEASUREMENT,MONTH_RECORDED,YEAR_RECORDED,MP_PRICE:_PRICE_PAID,SOURCE_SUPPLYING_PRICE_INFORMATION
0,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,1,2014,50.0,WFP
1,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,2,2014,50.0,WFP
2,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,3,2014,50.0,WFP
3,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,4,2014,50.0,WFP
4,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,5,2014,50.0,WFP


In [198]:
# create dataframe without year, month and price, to obtain one entry for each different shop
gfp_unique_shop = gfp[['COUNTRY_NAME','LOCALITY_NAME','MARKET_SHOPS']]
gfp_unique_shop.head()
initialSize = gfp_unique_shop.shape[0]
print("initial size", initialSize)
gfp_unique_shop.drop_duplicates(inplace=True)
finalSize = gfp_unique_shop.shape[0]
print("final size", finalSize)

initial size 743914
final size 1478


In [199]:
gfp_unique_shop.rename(columns={'MARKET_NAME':'MARKET_SHOPS'}, inplace=True) # renaming col in order to not misunderstand

In [200]:
list_all_countries = gfp_unique_shop.groupby(by='COUNTRY_NAME')['COUNTRY_NAME'].nunique()
print( '[COUNTRIES] Total Contries in the df', len(list_all_countries))

[COUNTRIES] Total Contries in the df 74


In [202]:
list_all_markets = gfp_unique_shop.groupby(by='MARKET_SHOPS')['MARKET_SHOPS'].nunique()
print('[MARKETS] Total markets in the df', len(list_all_markets))

[MARKETS] Total markets in the df 1449


In [214]:
#max
markets_all_country.max() # max instance for every col ## doesn't add much

MARKET_SHOPS    m. Kyiv
COUNT                19
dtype: object

In [215]:
#brand with more stores
markets_all_country.loc[markets_all_country['COUNT'] == markets_all_country['COUNT'].max()]

,MARKET_SHOPS,COUNT
0,National Average,19


In [216]:
#min
min_shops_per_brand = markets_all_country.loc[markets_all_country['COUNT'] == markets_all_country['COUNT'].min()]
min_shops_per_brand.head()

,MARKET_SHOPS,COUNT
12,Addis Ababa,1
13,Abala,1
14,Abalak,1
15,Abeche,1
16,Abengourou,1


In [217]:
#markts per country
mkt_per_country = gfp_unique_shop.groupby(by='MARKET_SHOPS')['COUNTRY_NAME'].value_counts()
mkt_per_country = mkt_per_country.to_frame()
mkt_per_country = mkt_per_country.rename(columns={mkt_per_country.columns[0]: 'COUNT'})
mkt_per_country.reset_index(inplace=True)
mkt_per_country.head()

,MARKET_SHOPS,COUNTRY_NAME,COUNT
0,Aba,Nigeria,1
1,Abaala,Ethiopia,1
2,Abala,Niger,1
3,Abalak,Niger,1
4,Abeche,Chad,1


In [ ]:
#finding empty

In [218]:
mkt_per_country.loc[mkt_per_country['MARKET_SHOPS'] == "National Average"]

,MARKET_SHOPS,COUNTRY_NAME,COUNT
1010,National Average,Armenia,1
1011,National Average,Azerbaijan,1
1012,National Average,Bhutan,1
1013,National Average,Chad,1
1014,National Average,Costa Rica,1
1015,National Average,Egypt,1
1016,National Average,El Salvador,1
1017,National Average,Guatemala,1
1018,National Average,Honduras,1
1019,National Average,India,1


In [188]:
#pdt per country
pdt_per_country = gfp.groupby(by='COMMODITY_PURCHASED')['COUNTRY_NAME'].value_counts()
pdt_per_country = pdt_per_country.to_frame()
pdt_per_country = pdt_per_country.rename(columns={pdt_per_country.columns[0]: 'COUNT'})
pdt_per_country.reset_index(inplace=True)
pdt_per_country.head()

,COMMODITY_PURCHASED,COUNTRY_NAME,COUNT
0,Apples,Kyrgyzstan,385
1,Apples,Algeria,80
2,Apples (red),Rwanda,599
3,Apples (red),Armenia,238
4,Apples (red),Gambia,170


In [ ]:
#finding empty

In [191]:
#pdt per mkt
pdt_per_mkt = gfp.groupby(by='COMMODITY_PURCHASED')['MARKET_SHOPS'].value_counts()
pdt_per_mkt = pdt_per_mkt.to_frame()
pdt_per_mkt = pdt_per_mkt.rename(columns={pdt_per_mkt.columns[0]: 'COUNT'})
pdt_per_mkt.reset_index(inplace=True)
pdt_per_mkt.head()

,COMMODITY_PURCHASED,MARKET_SHOPS,COUNT
0,Apples,Batken,38
1,Apples,Bishkek,38
2,Apples,Jalal-Abad,38
3,Apples,Kara-Balta,38
4,Apples,Karakol,38


In [ ]:
#finding empty

In [253]:
#NATIONAL_VALUE TO A NEW DF
nf_df = gfp.loc[gfp['MARKET_SHOPS'] == "National Average"]
nf_df.head()

,COUNTRY_NAME,LOCALITY_NAME,MARKET_SHOPS,COMMODITY_PURCHASED,NAME_OF_CURRENCY,MARKET_TYPE,UNIT_OF_GOODS_MEASUREMENT,MONTH_RECORDED,YEAR_RECORDED,MP_PRICE:_PRICE_PAID,SOURCE_SUPPLYING_PRICE_INFORMATION
11638,Armenia,NaN,National Average,Fuel (diesel),AMD,Retail,L,1,1996,100.0,WFP
11639,Armenia,NaN,National Average,Fuel (diesel),AMD,Retail,L,2,1996,100.0,WFP
11640,Armenia,NaN,National Average,Fuel (diesel),AMD,Retail,L,3,1996,110.0,WFP
11641,Armenia,NaN,National Average,Fuel (diesel),AMD,Retail,L,4,1996,100.0,WFP
11642,Armenia,NaN,National Average,Fuel (diesel),AMD,Retail,L,5,1996,110.0,WFP


In [251]:
new_gfp = gfp.copy()
new_gfp.loc[new_gfp['MARKET_SHOPS'] != "National Average"]
new_gfp.head()

,COUNTRY_NAME,LOCALITY_NAME,MARKET_SHOPS,COMMODITY_PURCHASED,NAME_OF_CURRENCY,MARKET_TYPE,UNIT_OF_GOODS_MEASUREMENT,MONTH_RECORDED,YEAR_RECORDED,MP_PRICE:_PRICE_PAID,SOURCE_SUPPLYING_PRICE_INFORMATION
0,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,1,2014,50.0,WFP
1,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,2,2014,50.0,WFP
2,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,3,2014,50.0,WFP
3,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,4,2014,50.0,WFP
4,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,5,2014,50.0,WFP


In [255]:
new_gfp = new_gfp.drop(labels='SOURCE_SUPPLYING_PRICE_INFORMATION', axis=1)


In [261]:
#new_df sem duplicados
new_gfp_product = new_gfp.copy
new_gfp_product = new_gfp[['COUNTRY_NAME', 'LOCALITY_NAME','MARKET_SHOPS', 'COMMODITY_PURCHASED']]
new_gfp_product = new_gfp_product.drop_duplicates()
new_gfp_product = new_gfp_product.reset_index(drop=True)
new_gfp_product.head()

,COUNTRY_NAME,LOCALITY_NAME,MARKET_SHOPS,COMMODITY_PURCHASED
0,Afghanistan,Badakhshan,Fayzabad,Bread
1,Afghanistan,Badakhshan,Fayzabad,Wheat
2,Afghanistan,Badakhshan,Fayzabad,Rice (low quality)
3,Afghanistan,Badakhshan,Fayzabad,Wage (qualified labour)
4,Afghanistan,Badakhshan,Fayzabad,"Livestock (sheep, one-year-old alive female)"


In [275]:
#sold products

count_prod = new_gfp_product['COMMODITY_PURCHASED'].count()
count_prod

16165

In [283]:
P_UNIQ = new_gfp_product['COMMODITY_PURCHASED'].unique()
len(P_UNIQ)

321

In [280]:
c_p = new_gfp_product['COMMODITY_PURCHASED'].value_counts()
c_p = c_p.to_frame()
c_p.reset_index(inplace=True)
c_p.rename(columns={c_p.columns[0]:'PRODUCTS',c_p.columns[1]:'QUANTITY_PRODS'}, inplace=True)
c_p.head()

,PRODUCTS,QUANTITY_PRODS
0,Maize,457
1,Rice (imported),446
2,Sorghum,430
3,Rice (local),419
4,Sugar,411
